In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from toolkit import slice_spectrum, concatenate_spectra, bands_TiO, match_spectra, SimpleSpectrum
import astropy.units as u
from scipy.optimize import fmin_l_bfgs_b

In [7]:
def plot_spliced_spectrum(observed_spectrum, model_flux, other_model=None):
    n_chunks = len(observed_spectrum.wavelength_splits)
    fig, ax = plt.subplots(n_chunks, 1, figsize=(8, 10))

    for i, inds in enumerate(observed_spectrum.wavelength_splits):
        min_ind, max_ind = inds
        
        ax[i].errorbar(observed_spectrum.wavelength[min_ind:max_ind].value, 
                       observed_spectrum.flux[min_ind:max_ind], 
                       0.025*np.ones(max_ind-min_ind))
        ax[i].plot(observed_spectrum.wavelength[min_ind:max_ind], 
                   model_flux[min_ind:max_ind])
        
        if other_model is not None:
            ax[i].plot(observed_spectrum.wavelength[min_ind:max_ind], 
                       other_model[min_ind:max_ind], alpha=0.4)
        
        ax[i].set_xlim([observed_spectrum.wavelength[min_ind].value,
                        observed_spectrum.wavelength[max_ind-1].value])
        ax[i].set_ylim([0.9*observed_spectrum.flux[min_ind:max_ind].min(), 
                        1.1])

    return fig, ax

In [8]:
import h5py

archive = h5py.File('/Users/bmmorris/git/aesop/notebooks/spectra.hdf5', 'r+')

In [9]:
list(archive['51Peg'])

['2017-09-05T06:04:52.570',
 '2017-09-05T06:09:54.789',
 '2017-09-11T01:56:48.809',
 '2017-09-11T02:02:20.940']

In [10]:
targets = list(archive)
spectrum1 = archive['KIC9652680']['2016-07-05T09:58:18.760']
wavelength1 = spectrum1['wavelength'][:]
flux1 = spectrum1['flux'][:]
target = SimpleSpectrum(wavelength1, flux1, dispersion_unit=u.Angstrom)

spec_band = []

first_n_bands = 5
width = 3

for band in bands_TiO[:first_n_bands]:
    target_slice = slice_spectrum(target, band.min-width*u.Angstrom, band.max+width*u.Angstrom)
    target_slice.flux /= target_slice.flux.max()
    spec_band.append(target_slice)

target_slices = concatenate_spectra(spec_band)

spectrum2 = archive['51Peg']['2017-09-11T01:56:48.809']#archive[target][time]
wavelength2 = spectrum2['wavelength'][:]
flux2 = spectrum2['flux'][:]
source1 = SimpleSpectrum(wavelength2, flux2, dispersion_unit=u.Angstrom)

spec_band = []
for band, inds in zip(bands_TiO[:first_n_bands], target_slices.wavelength_splits):
    target_slice = slice_spectrum(source1, band.min-width*u.Angstrom, band.max+width*u.Angstrom, 
                                  force_length=abs(np.diff(inds))[0])
    target_slice.flux /= target_slice.flux.max()
    spec_band.append(target_slice)

source1_slices = concatenate_spectra(spec_band)

In [17]:
def chi2(p, target, comp1, comp2):
    model, residuals = instr_model(target, comp1, comp2, *p)
    return residuals

In [25]:
from toolkit import instr_model
from astropy.utils.console import ProgressBar
bfgs_options_fast = dict(epsilon=1e-3, approx_grad=True,
                         m=10, maxls=20)
bfgs_options_precise = dict(epsilon=1e-3, approx_grad=True,
                            m=30, maxls=50)

target_list = []
time_list = []
residuals_list = []
area_list = []
counter = 0
for target in archive: 
    for time in archive[target]:
        counter +=1 

for target in archive: 
    for time in archive[target]:
        if target != 'KIC9652680' and target != '51Peg':

            spectrum3 = archive[target][time]
            wavelength3 = spectrum3['wavelength'][:]
            flux3 = spectrum3['flux'][:]
            source2 = SimpleSpectrum(wavelength3, flux3, dispersion_unit=u.Angstrom)

            spec_band = []
            for band, inds in zip(bands_TiO[:first_n_bands], target_slices.wavelength_splits):
                target_slice = slice_spectrum(source2, band.min-width*u.Angstrom, band.max+width*u.Angstrom, 
                                              force_length=abs(np.diff(inds))[0])
                target_slice.flux /= target_slice.flux.max()
                spec_band.append(target_slice)

            source2_slices = concatenate_spectra(spec_band)

            bounds = [[-30, 0], [0, 15]] + first_n_bands*[[-2, 2]]
            initp = [-0.5, 1] + first_n_bands*[0.0]

            result = fmin_l_bfgs_b(chi2, initp, bounds=bounds, 
                                   args=(target_slices, source1_slices, source2_slices),
                                   **bfgs_options_precise)

            model, resid = instr_model(target_slices, source1_slices, source2_slices, *result[0])

            target_list.append(target)
            time_list.append(time)
            residuals_list.append(resid)
            area_list.append(np.exp(result[0][0]))
            print(target, resid, np.exp(result[0][0]))

/Users/bmmorris/anaconda/lib/python3.5/site-packages/scipy/signal/windows.py:1217: RuntimeWarning: divide by zero encountered in true_divide
  w = np.exp(-n ** 2 / sig2)


GJ4099 0.273652152867 0.026156648259
GJ4099 0.376290139184 1.02404502467e-13
GJ702B 0.275963237122 0.0855269638279
GJ9781A 0.272833269119 0.130178276445
HATP11 0.278672932811 0.00019095726489
HATP11 0.278661499135 0.000208422637521
HATP11 0.278651446581 1.38773112835e-06
HATP11 0.278651400327 1.32475635597e-06
HATP11 0.278651523315 9.56440103778e-06
HATP11 0.278665132305 7.42036937231e-07
HATP11 0.334050056326 0.088977723921
HATP11 0.27865139539 6.23519836344e-07
HATP11 0.278665073365 1.20967661284e-06
HATP11 0.336609394055 0.000434148550282
HATP11 0.278651504294 3.33003944561e-06
HATP11 0.278712917853 0.00129382965263
HATP11 0.278651394502 4.23187343605e-07
HATP11 0.278665174804 1.41563484884e-05
HATP11 0.382699259367 0.0407034047833
HATP11 0.278677645683 0.000157068489392
HATP11 0.278665730009 6.53814327875e-06
HATP11 0.278665113991 1.42001232849e-07
HATP11 0.278651224195 9.35762296884e-14
HATP11 0.278651408928 1.09953441812e-09
HD10697 0.273263535426 0.304643786395
HD110833 0.276365

In [28]:
first_percentile = np.percentile(residuals_list, 2)

In [29]:
for target, time, resid, area in zip(target_list, time_list, residuals_list, area_list):
    if first_percentile > resid: 
        print(target, time, resid, area)

HD39587 2016-12-02T09:35:04.980 0.246809249948 0.420588836135
HD89744 2017-11-06T09:45:40.470 0.250203466225 0.516621318119
HD98230 2017-03-17T06:05:03.709 0.250254675589 0.358736673515
